In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 

from pycaret.classification import *
import os
import joblib
import numpy as np
import shutil

In [2]:
selected_descriptor = pd.read_csv('../data/descriptor_selection.csv')

file_md_list = {}
for column in selected_descriptor.columns:
    filename = column
    selected_columns = selected_descriptor[column].iloc[0:].dropna().tolist()
    if filename and selected_columns:
        file_md_list[filename] = selected_columns

In [ ]:
default_filenames = {
    'auc': 'AUC.png',
    'confusion_matrix': 'Confusion Matrix.png',
    'learning': 'Learning Curve.png',
    'feature': 'Feature Importance.png',
    'error': 'Prediction Error.png',
    'calibration': 'Calibration Curve.png'
}

for ratio in ['5x','10x','20x']:
    for TF in ['True','False']:
        file_name = f'descriptors_filtered_FTO_training_{ratio}_ignore3D_{TF}.csv'
        base_path = f'FTO_FP+weka_MD/{ratio}_md{TF}'.replace('True', 'T').replace('False', 'F')
        df = pd.read_csv(f"../data/preprocessed/filtered_FTO_training_{ratio}_ignore3D_{TF}.csv")
        print(f"{'='*50}{base_path} start{'='*50}")

        # 저장 폴더 생성
        os.makedirs(f'../result/{base_path}', exist_ok=True)
        os.makedirs(os.path.join("../result", base_path, "models"), exist_ok=True)
        plot_save_dir = os.path.join("../result", base_path, "plots")
        os.makedirs(plot_save_dir, exist_ok=True)

        md_cols = file_md_list[file_name]
        fp_cols = [f'X{i+1}' for i in range(1024)]
        filtered_df = df[['potency']+fp_cols+md_cols]

        X = filtered_df.drop('potency', axis=1)
        Y = filtered_df['potency']

        x_train, x_test, y_train, y_test = train_test_split(
            X, Y,
            test_size=0.1,
            random_state=42,
            stratify=Y
        )

        # 모델 구축
        all_comparison_df = {} 
        df_train = pd.concat([x_train, y_train], axis=1)

        setup(
            data=df_train,
            target='potency',
            session_id=42,
            train_size=0.9,
            fold=10,
            normalize=True,
            fix_imbalance=True,
            remove_outliers=True,
            n_jobs=-1, 
            verbose=False
        )
        compare_models(fold=5, sort='F1', n_select=5, turbo=True) 
        results_df = pull()

        # Top 5 모델 튜닝
        top_models = results_df.index[:5].tolist()
        summary_data = []

        for model_id in top_models:
            print(f"Tuning {model_id}...")
            
            # 모델 생성 및 튜닝
            model = create_model(model_id, verbose=False)
            tuned_model = tune_model(
                model, 
                optimize='F1',
                n_iter=50,
                fold=5,
                choose_better=True,
                verbose=False
            )
            
            # 결과 데이터 가져오기
            results = pull()
            model_name = tuned_model.__class__.__name__
            
            # 튜닝된 모델 저장 (.pkl)
            model_path = os.path.join("../result", base_path, "models", f"{ratio}_{model_name}_model.pkl")
            joblib.dump(tuned_model, model_path)
            
            # 평가 결과 저장 (.csv)
            eval_path = os.path.join("../result", base_path, f"{ratio}_{model_name}_evaluation.csv")
            results.to_csv(eval_path, index=False)
            
            # 각 모델별 plot 생성
            for plot_type, default_name in default_filenames.items():
                try:
                    plot_model(tuned_model, plot=plot_type, save=True, verbose=False)
                    
                    final_filename = f"{ratio}_{model_name}_{plot_type}.png"
                    final_save_path = os.path.join(plot_save_dir, final_filename)
                    
                    if os.path.exists(default_name):
                        shutil.move(default_name, final_save_path)
                        
                except Exception as e:
                    print(f"     - [{plot_type}] Plot 생성 실패: {e}")
            
            # 요약 데이터 추가
            numeric_cols = results.select_dtypes(include=[np.number]).columns
            avg_row = results[numeric_cols].mean().to_dict()
            std_row = results[numeric_cols].std().to_dict()
            
            avg_row.update({'Model': model_name, 'Type': 'Mean'})
            std_row.update({'Model': model_name, 'Type': 'Std'})
            
            summary_data.extend([avg_row, std_row])

        combined_summary = pd.DataFrame(summary_data)
        summary_path = os.path.join("../result", base_path, f"{ratio}_summary_evaluation.csv")
        combined_summary.to_csv(summary_path, index=False)

==================================================FTO_FP+weka_MD/5x_mdT start==================================================


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9483,0.9643,0.8009,0.8836,0.8387,0.8080,0.8105,0.9500
et,Extra Trees Classifier,0.9483,0.9604,0.7852,0.8942,0.8358,0.8052,0.8079,0.2460
gbc,Gradient Boosting Classifier,0.9457,0.9622,0.8113,0.8586,0.8329,0.8005,0.8019,0.8940
svm,SVM - Linear Kernel,0.9448,0.9602,0.7487,0.9101,0.8194,0.7873,0.7938,0.1920
lightgbm,Light Gradient Boosting Machine,0.9439,0.9606,0.7484,0.9024,0.8172,0.7844,0.7899,0.3960
rf,Random Forest Classifier,0.9422,0.9680,0.7538,0.8842,0.8133,0.7794,0.7832,0.2500
ada,Ada Boost Classifier,0.9247,0.9202,0.7490,0.7901,0.7682,0.7233,0.7242,0.3560
dt,Decision Tree Classifier,0.8905,0.8253,0.7274,0.6616,0.6899,0.6239,0.6271,0.2180
nb,Naive Bayes,0.8888,0.7156,0.4553,0.7956,0.5784,0.5196,0.5470,0.6500
ridge,Ridge Classifier,0.7530,0.8243,0.7745,0.3850,0.5131,0.3727,0.4143,0.1960


Tuning lr...
Tuning et...
Tuning gbc...
Tuning svm...
     - [auc] Plot 생성 실패: AUC plot not available for estimators with no predict_proba attribute.
     - [calibration] Plot 생성 실패: Calibration plot not available for estimators with no predict_proba attribute.
Tuning lightgbm...
==================================================FTO_FP+weka_MD/5x_mdF start==================================================


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9518,0.9620,0.8062,0.8966,0.8486,0.8200,0.8220,0.9700
lr,Logistic Regression,0.9492,0.9656,0.8063,0.8837,0.8422,0.8120,0.8140,0.2080
lightgbm,Light Gradient Boosting Machine,0.9483,0.9593,0.7695,0.9078,0.8324,0.8022,0.8063,0.4140
et,Extra Trees Classifier,0.9466,0.9673,0.7800,0.8885,0.8304,0.7988,0.8014,0.2640
rf,Random Forest Classifier,0.9422,0.9622,0.7540,0.8843,0.8135,0.7796,0.7833,0.2600
svm,SVM - Linear Kernel,0.9387,0.9545,0.7012,0.9136,0.7919,0.7568,0.7668,0.1980
ada,Ada Boost Classifier,0.9299,0.9195,0.7645,0.8075,0.7843,0.7426,0.7437,0.3620
dt,Decision Tree Classifier,0.8966,0.8373,0.7483,0.6764,0.7083,0.6459,0.6487,0.2080
nb,Naive Bayes,0.8879,0.7150,0.4553,0.7855,0.5763,0.5168,0.5425,0.2120
ridge,Ridge Classifier,0.7574,0.8303,0.7640,0.3896,0.5149,0.3763,0.4146,0.2040


Tuning gbc...
Tuning lr...
Tuning lightgbm...
Tuning et...
Tuning rf...
==================================================FTO_FP+weka_MD/10x_mdT start==================================================


...

Tuning lr...
Tuning et...
Tuning gbc...
Tuning lightgbm...
Tuning svm...
     - [auc] Plot 생성 실패: AUC plot not available for estimators with no predict_proba attribute.
     - [calibration] Plot 생성 실패: Calibration plot not available for estimators with no predict_proba attribute.
==================================================FTO_FP+weka_MD/20x_mdT start==================================================


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9860,0.9728,0.7947,0.8982,0.8431,0.8358,0.8376,0.9520
rf,Random Forest Classifier,0.9842,0.9732,0.7421,0.9119,0.8170,0.8089,0.8142,0.5460
et,Extra Trees Classifier,0.9830,0.9779,0.7579,0.8678,0.8081,0.7993,0.8018,0.5720
lr,Logistic Regression,0.9810,0.9626,0.8263,0.7867,0.8050,0.7950,0.7959,0.4140
gbc,Gradient Boosting Classifier,0.9815,0.9674,0.7842,0.8199,0.8012,0.7915,0.7920,3.1140
ridge,Ridge Classifier,0.9727,0.9478,0.8474,0.6705,0.7476,0.7334,0.7396,0.3640
lda,Linear Discriminant Analysis,0.9725,0.9477,0.8474,0.6681,0.7459,0.7316,0.7381,0.6720
ada,Ada Boost Classifier,0.9742,0.9331,0.7263,0.7329,0.7292,0.7157,0.7159,0.9540
dt,Decision Tree Classifier,0.9605,0.8367,0.7000,0.5682,0.6263,0.6057,0.6099,0.4600
svm,SVM - Linear Kernel,0.9712,0.9708,0.4632,0.8694,0.6042,0.5907,0.6225,0.3800


Tuning lightgbm...
Tuning rf...
Tuning et...
Tuning lr...
Tuning gbc...
==================================================FTO_FP+weka_MD/20x_mdF start==================================================


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9850,0.9694,0.7842,0.8864,0.8316,0.8238,0.8258,1.0300
lr,Logistic Regression,0.9822,0.9641,0.8158,0.8117,0.8134,0.8041,0.8042,0.4260
et,Extra Trees Classifier,0.9832,0.9677,0.7526,0.8768,0.8089,0.8002,0.8033,0.5980
rf,Random Forest Classifier,0.9830,0.9797,0.7368,0.8864,0.8033,0.7945,0.7990,0.5380
gbc,Gradient Boosting Classifier,0.9815,0.9673,0.7789,0.8239,0.8005,0.7908,0.7913,3.2700
ridge,Ridge Classifier,0.9747,0.9492,0.8474,0.6975,0.7631,0.7499,0.7550,0.3880
lda,Linear Discriminant Analysis,0.9747,0.9491,0.8474,0.6975,0.7631,0.7499,0.7550,0.7400
ada,Ada Boost Classifier,0.9765,0.9555,0.7526,0.7549,0.7531,0.7408,0.7411,0.9720
svm,SVM - Linear Kernel,0.9760,0.9718,0.5737,0.8795,0.6923,0.6805,0.6983,0.3800
dt,Decision Tree Classifier,0.9585,0.8207,0.6684,0.5525,0.6015,0.5799,0.5847,0.4840


Tuning lightgbm...
Tuning lr...
Tuning et...
Tuning rf...
Tuning gbc...
